In [1]:
import pandas as pd

allowed_langs = ["en", "ru", "de", "fr", "uk-UA", "zh"]

In [2]:
messages = pd.read_json(
    path_or_buf="/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/datasets/pure_datasets/oasst_ready/2023-04-12_oasst_ready.messages.jsonl.gz",
    lines=True,
)

In [3]:
messages[:3]

,message_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,synthetic,emojis,labels,detoxify,message_tree_id,tree_state,parent_id,rank
0,6ab24d72-0181-4594-a9cd-deaf170242fb,c3fe8c76-fc30-4fa7-b7f8-c492f5967d18,2023-02-05T14:23:50.983374+00:00,Can you write a short introduction about the r...,prompter,en,3,1.0,False,False,"{'+1': 10, '_skip_reply': 1, '_skip_ranking': 4}","{'spam': {'value': 0.0, 'count': 3}, 'lang_mis...","{'toxicity': 0.00044308538781400006, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,NaN,NaN
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,1.0,False,False,"{'+1': 3, '_skip_labeling': 1}","{'spam': {'value': 0.0, 'count': 3}, 'fails_ta...","{'toxicity': 0.000263965659542, 'severe_toxici...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,6ab24d72-0181-4594-a9cd-deaf170242fb,0.0
2,6708c47f-05c9-4346-b3d2-40b2bd24fde4,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T18:48:49.391686+00:00,Now explain it to a dog,prompter,en,3,1.0,False,False,NaN,"{'spam': {'value': 0.0, 'count': 3}, 'lang_mis...","{'toxicity': 0.036484777927398, 'severe_toxici...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,c8e83833-ecbc-44fe-b6db-735228c25a1c,NaN


In [4]:
messages_dict = {}

for i in range(len(messages)):
    mess_id = messages.iloc[i]["message_id"]
    messages_dict[mess_id] = messages.iloc[i].to_dict()

In [8]:
trees = pd.read_json(
    path_or_buf="/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/datasets/pure_datasets/oasst_ready/2023-04-12_oasst_ready.trees.jsonl.gz",
    lines=True,
)
trees
trees_dict = {}
for i in range(len(trees)):
    tree_id = trees["message_tree_id"][i]
    trees_dict[tree_id] = trees["prompt"][i]

In [ ]:
example_tree_id = list(trees_dict.keys())[3]

trees_dict[example_tree_id]

In [14]:
filtered_trees_dict = {}

for tree_id in trees_dict:
    tree_lang = trees_dict[tree_id]["lang"]
    if tree_lang in ["ru"]:
        filtered_trees_dict[tree_id] = trees_dict[tree_id]

In [27]:
len(filtered_trees_dict)

760

In [22]:
filtered_example_id = list(filtered_trees_dict.keys())[31]
filtered_example = filtered_trees_dict[filtered_example_id]
# filtered_example

In [ ]:
from collections.abc import Iterable


def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x


# https://codebeautify.org/python-formatter-beautifier - for debug
def get_all_threads(sample):
    # debug
    # initial_message = f"{sample['role'].upper()} {sample['text']} |"
    initial_message = f"{sample['message_id']} "

    def recursive_helper(node, current_path):
        if len(node["replies"]) == 0:
            if node["role"] == "assistant":
                # debug
                # current_path += f"\n|||||||||||||\n {node['role'].upper()} {node['text']}"
                current_path += f" {node['message_id']}"

            return current_path + " <END>"

        current_path += f" {node['message_id']}"
        paths = []
        for reply in node["replies"]:
            if node["labels"]["spam"]["value"] < 0.4:
                path = recursive_helper(reply, current_path)
                paths.append(path)
        return paths

    threads = []
    for reply in sample["replies"]:
        path = recursive_helper(reply, initial_message)
        threads.append(path)

    threads = flatten(threads)
    conversations = []
    for thread in threads:
        if "<END>" in thread:
            # debug
            # pprint.pprint(thread)
            # print(thread.split())
            conversation = thread.split()[:-1]
            conversations.append(conversation)

    return conversations


def ids_to_text(message_ids, messages):
    chat_history = []
    for mess_id in message_ids:
        text = messages[mess_id]["text"]
        chat_history.append(text)

    return chat_history


# all_threads = get_all_threads(sample=trees["prompt"][10])
all_threads = get_all_threads(sample=filtered_example)
all_threads

In [ ]:
for thread in all_threads:
    messages_texts = ids_to_text(thread, messages_dict)
    print("\n---\n".join(messages_texts))
    print("-" * 100)

### prepare for filter export


In [34]:
import pandas as pd

export_name = "./datasets/filtered_datasets/oasst/ru/oasst_ru_v1.csv"
messages_count = 0

result_dataset = []

for filtered_id in filtered_trees_dict.keys():
    filtered_tree = filtered_trees_dict[filtered_id]
    filtered_threads_ids = get_all_threads(sample=filtered_tree)
    messages_count += len(filtered_threads_ids)
    for thread in filtered_threads_ids:
        thread_str = " ".join(thread)
        filtered_threads_mess = ids_to_text(thread, messages_dict)
        filtered_threads = "\n---\n".join(filtered_threads_mess)
        # print(filtered_threads)
        result_dataset.append(
            {
                "conversation": filtered_threads,
                "conversation_ids": thread_str,
            },
        )
        # print("-" * 100)
    # print("-" * 100)
    # print("-" * 100)
    # print("-" * 100)
    # break
# messages_count
pd.DataFrame(data=result_dataset).to_csv(export_name, index=False)